# **Projeto: Consulta e análise dados com SQL**

Caderno de **Portfolio**<br>
[Christian Oliveira](https://www.linkedin.com/in/christian-oliveira-925532257/) - [Github](https://github.com/VariableBee)

---

## **Tópicos**

<ol type="1">
  <li>Introdução;</li>
  <li>Análise Exploratória de Dados;</li>
  <li>Insights e Conclusão;</li>
</ol>

## **1\. Introdução**

- **Etapas**:
  - Criação do Bucket no AWSS3;
  - Criação da tabela AWS Athena;
  - SQL Querys ao banco de dados;
  - Análise e Storytelling;
  - Conclusão.

- **Fontes**:
  - A tabela foi criada no **AWS Athena** junto com o **S3 Bucket** com uma versão dos dados disponibilizados em: ([link](https://github.com/andre-marcos-perez/ebac-course-utils/tree/main/dataset));

### **Objetivo**

Este estudo tem como principal motivação compreender o perfil dos clientes de cartão de crédito em nossa base de dados e identificar padrões significativos que possam guiar a tomada de decisões estratégicas.

## **Os dados:**

Os dados representam informações de clientes de um banco e contam com as seguintes colunas:

* idade = idade do cliente
* sexo = sexo do cliente (F ou M)
* dependentes = número de dependentes do cliente
* escolaridade = nível de escolaridade do clientes
* salario_anual = faixa salarial do cliente
* tipo_cartao = tipo de cartao do cliente
* qtd_produtos = quantidade de produtos comprados nos últimos 12 meses
* iteracoes_12m = quantidade de iterações/transacoes nos ultimos 12 meses
* meses_inativo_12m = quantidade de meses que o cliente ficou inativo
* limite_credito = limite de credito do cliente
* valor_transacoes_12m = valor das transações dos ultimos 12 meses
* qtd_transacoes_12m  = quantidade de transacoes dos ultimos 12 meses







## **2\. Análise Exploratória de Dados**

### **Exploração**

A primeira fase da análise é a **exploração**. Assim podemos entender melhor os dados que estamos trabalhando com, fazendo perguntas aos dados:

**Qual a quantidade de informações temos na nossa base de dados?**
```
Query: SELECT count() FROM credito

> Reposta: 2564 linhas
```



Vamos trabalhar com uma fração dos dados. Na prática, quanto maior a quantidade de dados utilizada, mais confiável a análise! Mas os recursos computacionais e financeiros devem ser levados em consideração em cada caso. Como essa é uma análise para fins de estudo, a redução do volume de dados se torna interessante.

```
Query: SELECT * FROM credito LIMIT 10;

|  #  | idade | sexo | dependentes | escolaridade        | estado_civil | salario_anual      | tipo_cartao | qtd_produtos | interaces_12m | meses_inativo | limite_credito | valor_transacoes_12m | qtd_transacoes |
|----|-------|------|-------------|---------------------|--------------|--------------------|-------------|--------------|---------------|--------------|----------------|----------------------|----------------|
|  1 |  45   |  M   |     3       | ensino medio        | casado       | $60K - $80K        | blue        |     5        |      3        |      1       |   12691.51     |        1144.9         |       42       |
|  2 |  49   |  F   |     5       | mestrado            | solteiro     | menos que $40K     | blue        |     6        |      2        |      1       |    8256.96     |       1291.45         |       33       |
|  3 |  51   |  M   |     3       | mestrado            | casado       | $80K - $120K       | blue        |     4        |      0        |      1       |    3418.56     |       1887.72         |       20       |
|  4 |  40   |  F   |     4       | ensino medio        | na           | menos que $40K     | blue        |     3        |      1        |      4       |    3313.03     |       1171.56         |       20       |
|  5 |  40   |  M   |     3       | sem educacao formal | casado       | $60K - $80K        | blue        |     5        |      0        |      1       |    4716.22     |        816.08         |       28       |
|  6 |  44   |  M   |     2       | mestrado            | casado       | $40K - $60K        | blue        |     3        |      2        |      1       |    4010.69     |       1088.07         |       24       |
|  7 |  51   |  M   |     4       | na                  | casado       | $120K +            | gold        |     6        |      3        |      1       |   34516.72     |       1330.87         |       31       |
|  8 |  32   |  M   |     0       | ensino medio        | na           | $60K - $80K        | silver      |     2        |      2        |      2       |   29081.49     |       1538.32         |       36       |
|  9 |  37   |  M   |     3       | sem educacao formal | solteiro     | $60K - $80K        | blue        |     5        |      0        |      2       |   22352.5      |       1350.14         |       24       |
| 10 |  48   |  M   |     2       | mestrado            | solteiro     | $80K - $120K       | blue        |     6        |      3        |      3       |   11656.41     |       1441.73         |       32       |

```

Existem dados faltantes na tabela. Vamos considerar como e se há necessidade de tratá-los, mas primeiro:

**Quais os tipos de cada dado?**

```
Query: DESCRIBE credito

| Coluna              | Tipo     |
|---------------------|----------|
| idade               | int      |
| sexo                | string   |
| dependentes         | int      |
| escolaridade        | string   |
| estado_civil        | string   |
| salario_anual       | string   |
| tipo_cartao         | string   |
| qtd_produtos        | bigint   |
| interaces_12m       | int      |
| meses_inativo       | int      |
| limite_credito      | float    |
| valor_transacoes_12m| float    |
| qtd_transacoes      | int      |
```

Vamos explorar melhor cada tipo de dado pensando em como tratar os dados faltantes de cada tipo.

**Quais variáveis não numéricas que possuem dados faltantes?**

```
Query: SELECT DISTINCT escolaridade FROM credito

|  #  |   escolaridade         |
|----|-------------------------|
|  1 | mestrado                |
|  2 | graduacao               |
|  3 | doutorado               |
|  4 | sem educacao formal     |
|  5 | ensino medio            |
|  6 | na                      |

> Valores nulos (na) identificados em escolaridade.
```

```
Query: SELECT DISTINCT estado_civil  FROM credito

|  # | estado_civil |
|----|---------------|
|  1 | na            |
|  2 | casado        |
|  3 | divorciado    |
|  4 | solteiro      |

> Novamente encontramos valores nulos nos dados de estado civil.
```

```
Query: SELECT DISTINCT salario_anual  FROM credito

|  # |     salario_anual     |
|----|-----------------------|
|  1 |     $80K - $120K      |
|  2 |     $60K - $80K       |
|  3 |     $40K - $60K       |
|  4 |     $120K +           |
|  5 |     menos que $40K    |
|  6 |     na                |


>Também contem dados nulos.
```

**Ps:** Os salários nesse dataset não estão expostas com o valor exato que o cliente ganha. O que está informado é a faixa salarial de cada um. Isso de certa forma pode ser vantajoso para definirmos um público alvo por faixa salarial.

**Quais são os tipos de cartão disponíveis no dataset?**
```
Query: SELECT DISTINCT tipo_cartao  FROM credito

|  #  | tipo_cartao |
|----|------------|
|  1 | platinum   |
|  2 | silver     |
|  3 | blue       |
|  4 | gold       |


> Aqui vemos que não há necessidade de tratar valores nulos.
```

## **Análise de dados**


Agora que estamos familiarizados com nossos dados, podemos analisar as informações. Vamos fazer perguntas:

**Quantos clientes temos de cada faixa salarial?**

```
Query: select count(*), salario_anual from credito group by salario_anual

| _col0 | salario_anual   |
|-------|-----------------|
| 451   | $60K - $80K     |
| 467   | $40K - $60K     |
| 222   | $120K +         |
| 488   | $80K - $120K    |
| 701   | menos que $40K  |
| 235   | na              |

```
Vamos analisar esses dados por um momento


- Alta Renda: 710 clientes recebem mais de $80.000

- Renda média: 918 clientes recebem entre $40/80k

- Baixa renda: 701 clientes recebem abaixo dos $40.000

Desconsiderando os 235 clientes que não informaram ou não consta a faixa salarial.

Essa informação pode ser **bastsante valiosa**. Baseado na renda é possível definir aqueles clientes como público alvo para uma determinada campanha, estratégia, ou objetivo da empresa.










**Qual a proporção de clientes do sexo masculino/feminino?**
```
Query: select count(*), sexo from credito group by sexo

| _col0 | sexo |
|-------|------|
| 1563  | M    |
| 1001  | F    |


> A maioria dos clientes é do sexo masculino.
```

**Qual é a idade média dos clientes por sexo?**
```
Query: select avg(idade) as media_idade, min(idade) as min_idade, max(idade) as max_idade, sexo from credito group by sexo

| # | media_idade        | min_idade | max_idade | sexo |
|---|--------------------|-----------|-----------|------|
| 1 | 46.42957042957043 | 26        | 67        | F    |
| 2 | 45.57389635316699 | 26        | 73        | M    |


> Por meio dessa análise não foi possível extrair nenhuma informação relevante para a empresa.
```

**Mas e quanto a idade média por salário?**
```
Query: SELECT idade, AVG(CAST(salario_anual AS DOUBLE)) AS salario_medio FROMcredito
GROUP BY idade;

salario_anual      | media_idade
-------------------|-------------
menos que 40K      | 45
40K - 60K          | 45
60K - 80K          | 45
80K - 120K         | 46
120K +             | 47
na                 | 45

> Nada muito conclusivo também.
```

**Ps:** Vamos faser as próximas querys já lidando com os dados faltantes.

**Quais as características dos clientes que possuem os maiores creditos?**
```
Query:
select max(limite_credito) as limite_credito, escolaridade, tipo_cartao, sexo
from credito where escolaridade != 'na' and tipo_cartao != 'na'group by  escolaridade, tipo_cartao, sexo order by limite_credito desc limit 10

| limite_credito | escolaridade       | tipo_cartao | sexo |
|----------------|--------------------|-------------|------|
| 34516.99       | sem educacao formal | blue        | M    |
| 34516.98       | mestrado            | gold        | M    |
| 34516.97       | mestrado            | blue        | M    |
| 34516.96       | sem educacao formal | silver      | M    |
| 34516.96       | doutorado           | platinum    | F    |
| 34516.95       | ensino medio        | gold        | M    |
| 34516.94       | graduacao           | gold        | F    |
| 34516.94       | graduacao           | gold        | M    |
| 34516.89       | graduacao           | silver      | M    |
| 34516.74       | ensino medio        | blue        | M    |

> Não parece haver um impacto da escolaridade no limite.
> O limite mais alto é oferecido para um cliente do sexo masculino sem educação formal.
> O cartão também parece não estar relacionado com a escolaridade nem com o limite.
> Dentre os maiores limites, encontramos clientes com cartão: gold, silver, platinum e blue
```


**Quais as características dos clientes que possuem os menores creditos?**
```
Query
select max(limite_credito) as limite_credito, escolaridade, tipo_cartao, sexo
from credito
where escolaridade != 'na' and tipo_cartao != 'na'
group by  escolaridade, tipo_cartao, sexo
order by limite_credito asc

|   # | limite_credito   | escolaridade        | tipo_cartao   | sexo   |
|---:|:-----------------|:--------------------|:--------------|:-------|
|  1 | 13,882.37        | ensino medio         | silver        | F      |
|  2 | 15,534.85        | ensino medio         | gold          | F      |
|  3 | 15,987.78        | mestrado             | gold          | F      |
|  4 | 23,032.31        | graduacao            | blue          | F      |
|  5 | 24,936.22        | mestrado             | blue          | F      |
|  6 | 26,053.49        | doutorado            | blue          | F      |
|  7 | 26,181.89        | sem educacao formal  | blue          | F      |
|  8 | 29,543.10        | ensino medio         | blue          | F      |
|  9 | 30,310.45        | mestrado             | silver        | F      |
| 10 | 34,516.16        | doutorado            | gold          | M      |

> Dessa vez conseguimos perceber que não há clientes com cartão platinum dentre os menores limites.
> Também foi possível perceber que a maioria dos menores limites são do sexo feminino enquanto nos maiores limites predomina o masculino.
```

**Quais as principais características dos clientes por faixa salarial?**
```
|   | salario_anual   | idade_media   | escolaridade_mais_comum   | sexo_mais_comum   |
|---|-----------------|---------------|---------------------------|-------------------|
| 1 | $120K +         | 47.87         | sem educacao formal       | M                 |
| 2 | $40K - $60K     | 45.65         | sem educacao formal       | M                 |
| 3 | $60K - $80K     | 45.26         | sem educacao formal       | M                 |
| 4 | $80K - $120K    | 46.17         | sem educacao formal       | M                 |
| 5 | menos que $40K  | 45.73         | sem educacao formal       | M                 |

> Já haviamos observado que a idade não é tão conclusiva e que a maioria dos clientes é do sexo masculino.
> Vamos analisar a escolaridade mais de perto.
```

A escolaridade mais comum em todas as faixas salariais é "sem educação formal". No entanto, isso não significa que ter uma escolaridade não impacte no salário. Mas para investigarmos isso teremos de calcular uma média para as faixas salariais, porém elas não estão em decimais, então vamos "assumir" uma média.

**A escolaridade afeta o salário?**
```
Query SELECT
  escolaridade,
  AVG(CASE
    WHEN salario_anual = 'menos que $40K' THEN 30000
    WHEN salario_anual = '$40K - $60K' THEN 50000
    WHEN salario_anual = '$60K - $80K' THEN 70000
    WHEN salario_anual = '$80K - $120K' THEN 100000
    WHEN salario_anual = '$120K +' THEN 120000
    ELSE 0
  END) AS media_salario_aprox
FROM credito
WHERE escolaridade != 'na' AND salario_anual != 'na'
GROUP BY escolaridade
ORDER BY media_salario_aprox DESC;

|    | escolaridade         | media_salario_aprox   |
|---:|:---------------------|-----------------------|
|  1 | graduacao            | 68,230.45             |
|  2 | ensino medio         | 65,818.18             |
|  3 | doutorado            | 64,759.62             |
|  4 | mestrado             | 64,000.00             |
|  5 | sem educacao formal  | 62,598.19             |

> Observe que a média de salario aproximada, para a maioria dos clientes sem educação formal, é a mais baixa.
> No geral, é inconclusivo se existe uma relação entre escolaridade e salário.
```



Para uma análise mais completa e precisa da relação entre idade e escolaridade com o salário, seria necessário levar em conta outros fatores, como experiência profissional, setor de trabalho, localização geográfica e demanda por habilidades específicas. Além disso, uma análise estatística mais avançada, como uma regressão linear, poderia ser aplicada para quantificar a influência da escolaridade no salário, controlando outros fatores.

**Será que as mulheres gastam mais?**
```
Query
select max(valor_transacoes_12m) as maior_valor_gasto, avg(valor_transacoes_12m) as  media_valor_gasto, min(valor_transacoes_12m) as min_valor_gasto, sexo
from credito
group by sexo

|   # | maior_valor_gasto   | media_valor_gasto   | min_valor_gasto   | sexo   |
|---:|:---------------------|:--------------------|:------------------|:-------|
|  1 | 4,686.93             | 1,807.97            | 530.36            | M      |
|  2 | 4,776.58             | 1,839.62            | 510.16            | F      |

> Apesar da diferença nos limites, os gastos são muito próximos.
```

**Por fim, o salário impacta no limite?**
```
Query
select avg(qtd_produtos) as qts_produtos, avg(valor_transacoes_12m) as media_valor_transacoes, avg(limite_credito) as media_limite,  sexo,   salario_anual from credito
where salario_anual != 'na'
group by sexo, salario_anual
order by avg(valor_transacoes_12m) desc

|   # | qtd_produtos  | media_valor_transacoes   | media_limite   | sexo   |  salario_anual  |
|---:|:---------------|:-------------------------|:---------------|:-------|:----------------|
|  1 | 4.39           | 1,963.67                 | 3,452.40       | M      | menos que $40K  |
|  2 | 4.29           | 1,878.30                 | 5,753.31       | M      | $40K - $60K     |
|  3 | 4.41           | 1,845.13                 | 4,212.20       | F      | menos que $40K  |
|  4 | 4.35           | 1,818.64                 | 9,096.03       | M      | $60K - $80K     |
|  5 | 4.44           | 1,781.43                 | 4,773.45       | F      | $40K - $60K     |
|  6 | 4.33           | 1,755.25                 | 14,886.56      | M      | $80K - $120K    |
|  7 | 4.33           | 1,701.47                 | 17,801.49      | M      | $120K +         |

>As pessoas que tem a menor faixa salarial também apresentam menor limite de credito!
```

# Conclusão

Essas foram **algumas** análises extraídas do dataset de crédito.  

Alguns insights interessantes:

- A maior parte dos clientes(1.628) possui renda anual acima dos 40 mil dólares. Mas estão destribuidos em diferentes faixas salariais.
- A maior concentração de clientes em uma única faixa salarial é de 701 clientes, que recebem abaixo dos $40.000.
- A idade e escolaridade **aparentam**¹ não influenciar no limite, salario anual e nem no tipo do cartão.
- Os clientes com maiores limites são em sua maioria do sexo masculino.
- Os clientes com menores limites são em sua maioria do sexo feminino.
- Dentre os menores limites não há presença de cartão platinum
- A faixa salarial impacta diretamente no limite de crédito
- Não existem clientes com salário anual acima de 60K do sexo feminino

**Nota¹:** Para uma análise mais completa e precisa da relação entre idade e escolaridade com o salário, seria necessário levar em conta outros fatores, como experiência profissional, setor de trabalho, localização geográfica e demanda por habilidades específicas. Além disso, uma análise estatística mais avançada, como uma regressão linear, poderia ser aplicada para quantificar a influência da escolaridade no salário, controlando outros fatores.

Poderia ser interessante para a empresa desenvolver campanhas e estratégias focadas no grupo de clientes que estão na menor faixa salarial. Sabemos que em sua maioria são clientes do sexo feminino, a média de suas idades, e que recebem os menores salários e tem o menor crédito.

Os dados nos mostram um possível reflexo cultural da nossa sociedade se refletindo na desigualdade de salários entre homens e mulheres. Mas isso também requer uma análise mais extensa como observei na **Nota¹**.
